# Connectome analysis
## Set parameters

In [ ]:
EXPERIMENT_DIRECTORY = "rebased_on_mjd"                     # e.g. "rebased_on_mjd", "p4", ...
USE_LOCAL_DATA = True                                       # if False, it tries to read the data on the laboratory's server
IS_COLLABORATION_PROJ = True
import os
COLLABORATION_DIRECTORY = os.path.join("Mathias Schmidt", "soumnya")

# ###################################### LOCAL DIRECTORIES ######################################
DATA_ROOT  = f"../data/experiments/soumnya/{EXPERIMENT_DIRECTORY}"
PLOTS_ROOT = f"../plots/soumnya/{EXPERIMENT_DIRECTORY}/"
# DATA_ROOT  = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
# PLOTS_ROOT = f"../plots/{EXPERIMENT_DIRECTORY}"

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
BRANCHES_TO_EXCLUDE = ["retina", "VS", "grv", "fiber tracts", "CB"]
USE_LITERATURE_REUNIENS = False                             # add a 'REtot' region, merging the following regions: 'PR', 'RE', 'Xi', 'RH'
# MIN_AREA = 0.0                                            # area in mm². If a region of one animal is smaller, that same animal won't be displayed in the plots
MIN_AREA = 0.1 # we used this for bar plots
NORMALIZATION = "Density"                                   # call get_normalization_methods() on a AnimalGroup object to know its available normalization methods
REGIONS_TO_PLOT_SELECTION_METHOD = "summary structures"     # Available options are:
                                                            #   - "summary structures"
                                                            #   - "nre to bla"
                                                            #   - "major divisions"
                                                            #   - "depth <n>" where <n> is an integer of the depth desired
                                                            #   - "structural level <n>" where <n> is an integer of the level desired
                                                            #   - "pls <experiment> <salience_threshold>" (e.g., "pls proof 1.2")
SAVED_PLOT_EXTENSION = ".html"                              # '.html' for interactive plot
                                                            # '.svg' for vectorized image
                                                            # '.png'/'.jpg'/... for rasterized image
# ###################################### CROSS CORRELATION ######################################
MIN_ANIMALS_CROSS_CORRELATION = 4                           # 'None' means that ALL the animals must have the region, otherwise the correlation is NaN
PLOT_ONLY_REGIONS_PRESENT_IN_ALL_GROUPS = False             # if False, the correlation matrix and the chord plots of two groups may refer to different regions
PLOT_REGIONS_WITH_INSUFFICIENT_DATA = True                  # what to do with brain regions with less animals than min_animals_cross_correlation

# ###################################### CORRELATION MATRIX #####################################
MATRIX_SAVE_PLOT = False
MATRIX_SHOW_PLOT = False
MATRIX_CELL_HEIGHT = 18
MATRIX_STAR_SIZE = 8
MATRIX_CELL_RATIO = 1
MATRIX_MIN_PLOT_HEIGHT = 500

# ######################################## CORR. NETWORK ########################################
# NETWORK_P_CUTOFF = 0.05                                     # 1 if you don't want to filter by p-value
NETWORK_P_CUTOFF = 0.01
# NETWORK_P_CUTOFF = 0.0025
NETWORK_R_CUTOFF = 0.87
NETWORK_USE_NEGATIVE_LINKS = False
NETWORK_USE_ISOLATED_VERTICES = False

# ######################################## CHORD DIAGRAM ########################################
CHORD_SAVE_PLOT = False
CHORD_SHOW_PLOT = True
CHORD_PLOT_SIZE = 1200
CHORD_NO_BACKGROUND = False
CHORD_REGIONS_SIZE = 10
CHORD_REGIONS_FONT_SIZE = 10
CHORD_MAX_EDGE_WIDTH = 3
CHORD_USE_WEIGHTED_EDGE_WIDTHS = False
CHORD_USE_COLORSCALE_EDGES = True
CHORD_COLORSCALE = "Plasma"                                 # see https://plotly.com/python/builtin-colorscales/
CHORD_COLORSCALE_MIN = "cutoff"
CHORD_BOTTOM_ANNOTATIONS = dict(
    annotation1 = "Dark grey nodes are regions with insufficient data to compute cross correlation",
    annotation2 = "Light grey nodes are regions with no correlation with others above the threshold",
    annotation3 = "This is the third annotation",
    # howmany annotations desired with the following format:
    # annotations<k> = "<annotation>"
)
# ###############################################################################################
from plotly.colors import DEFAULT_PLOTLY_COLORS
from collections import namedtuple
GroupInfo = namedtuple("GroupInfo", "name colour")

In [ ]:
# SHILA - 3 Groups {Control|Stress|Resilient}
groups = [
    GroupInfo(
        name="Control",
        colour=DEFAULT_PLOTLY_COLORS[6]
    ),
    GroupInfo(
        name="Stress",
        colour=DEFAULT_PLOTLY_COLORS[7]
    ),
    GroupInfo(
        name="Resilient",
        colour=DEFAULT_PLOTLY_COLORS[8]
    )
]
group_folder = "C-S-R"

In [ ]:
# SOUMNYA FEMALES+MALES - 2 Groups {Stress|Control}
# SHILA - 2 Groups {Control|Stress+Resilient}
groups = [
    GroupInfo(
        name="Control",
        colour=DEFAULT_PLOTLY_COLORS[4]
    ),
    GroupInfo(
        name="Stress",
        colour=DEFAULT_PLOTLY_COLORS[5]
    )
]
group_folder = "C-S"

In [ ]:
# SOUMNYA ALL - 2 Groups {Stress|Control} + 2 Groups {Males|Females}
groups = [
    GroupInfo(
        name="Control (Females)",
        colour=DEFAULT_PLOTLY_COLORS[0]
    ),
    GroupInfo(
        name="Stress (Females)",
        colour=DEFAULT_PLOTLY_COLORS[1]
    ),
    GroupInfo(
        name="Control (Males)",
        colour=DEFAULT_PLOTLY_COLORS[2]
    ),
    GroupInfo(
        name="Stress (Males)",
        colour=DEFAULT_PLOTLY_COLORS[3]
    )
]
group_folder = "CF-SF-CM-SM"

In [ ]:
# SOUMNYA FEMALES - 2 Groups {Stress|Control}
groups = [
    GroupInfo(
        name="Control (Females)",
        colour=DEFAULT_PLOTLY_COLORS[0]
    ),
    GroupInfo(
        name="Stress (Females)",
        colour=DEFAULT_PLOTLY_COLORS[1]
    )
]
group_folder = "CF-SF"

In [ ]:
# P4 - 2 Groups {Sham|SNi}
groups = [
    GroupInfo(
        name="SNi",
        colour=DEFAULT_PLOTLY_COLORS[0]
    ),
    GroupInfo(
        name="Sham",
        colour=DEFAULT_PLOTLY_COLORS[1]
    )
]
group_folder = "Sham-SNi"

In [ ]:
# SOUMNYA MALES - 2 Groups {Stress|Control}
groups = [
    GroupInfo(
        name="Control (Males)",
        colour=DEFAULT_PLOTLY_COLORS[2]
    ),
    GroupInfo(
        name="Stress (Males)",
        colour=DEFAULT_PLOTLY_COLORS[3]
    )
]
group_folder = "CM-SM"

## Script's code
run all cell below

In [ ]:
import os
import sys
import random

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
if not USE_LOCAL_DATA:
    match sys.platform:
        case "darwin":
            mnt_point = "/Volumes/Ricerca/"
            
        case "linux":
            mnt_point = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/"
        case "win32":
            mnt_point = "\\\\ich.techosp.it\\Ricerca\\"
        case _:
            raise Exception(f"Can't find the 'Ricerca' folder in the server for '{sys.platform}' operative system. Please report the developer (Carlo)!")
    if not os.path.isdir(mnt_point):
        raise Exception(f"Could not read '{mnt_point}'. Please be sure you are connected to the server.")
    if IS_COLLABORATION_PROJ:
        DATA_ROOT  =  os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", COLLABORATION_DIRECTORY, "data", EXPERIMENT_DIRECTORY)
        PLOTS_ROOT = os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", "Mathias Schmidt", "soumnya", "results", EXPERIMENT_DIRECTORY, "plots")
    else:
        DATA_ROOT  =  os.path.join(mnt_point, "Lab Matteoli", "Silva", "projects", "data", EXPERIMENT_DIRECTORY)
        PLOTS_ROOT = os.path.join(mnt_point, "Lab Matteoli", "Silva", "projects", "results", EXPERIMENT_DIRECTORY, "plots")

data_input_path     = os.path.join(DATA_ROOT, "BraiAn_output")
data_output_path    = os.path.join(data_input_path, group_folder)
plots_output_path   = os.path.join(PLOTS_ROOT, group_folder)

if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
BraiAn.cache(path_to_allen_json, "http://api.brain-map.org/api/v2/structure_graph_download/1.json")
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, BRANCHES_TO_EXCLUDE, use_literature_reuniens=USE_LITERATURE_REUNIENS)

In [ ]:
animal_groups: list[BraiAn.AnimalGroup] = []
for group in groups:
    animal_group = BraiAn.AnimalGroup.from_csv(group.name, data_input_path, f"cell_counts_{group.name}.csv")
    animal_groups.append(animal_group)
    print(f"Group '{animal_group.name}' - #animals: {animal_group.n}, marker: {animal_group.marker}")


In [ ]:
match REGIONS_TO_PLOT_SELECTION_METHOD:
    case "summary structures":
        # selects the Summary Strucutures
        path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
        AllenBrain.select_from_csv(path_to_summary_structures, include_nre_tot=USE_LITERATURE_REUNIENS)
    case "nre to bla":
        # selects the NRe to BLA inputs
        path_to_inputs = os.path.join(project_path, "data", "NRe_to_BLA_inputs.csv")
        AllenBrain.select_from_csv(path_to_inputs, include_nre_tot=USE_LITERATURE_REUNIENS)
        nre_bla_regions = AllenBrain.get_selected_regions()
        nre_bla_regions += ("REtot", "BLA")
        AllenBrain.select_regions(nre_bla_regions)
    case "major divisions":
        AllenBrain.select_regions(BraiAn.MAJOR_DIVISIONS)
    case s if s.startswith("pls"):
        options = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")
        assert len(options) == 3, "The 'REGIONS_TO_PLOT_SELECTION_METHOD' option is invalid. Make sure it follows the follows the following pattern: \"pls <experiment> <salience_threshold>\" (e.g., \"pls proof 1.2\")"
        pls_experiment, pls_threshold = options[1:]
        pls_threshold = pls_threshold.replace(".", "_")
        assert len(animal_groups) == 2, f"You can't use the PLS of '{pls_experiment}' for selecting the regions to plot because '{group_folder}' has too many groups ({len(animal_groups)})"
        pls_file = f"pls_{animal_groups[0].marker}_{NORMALIZATION}_salient_regions_above_{pls_threshold}.csv".lower()
        regions_to_plot_pls_csv = os.path.abspath(os.path.join(DATA_ROOT, os.pardir, pls_experiment, "BraiAn_output", group_folder, pls_file))
        assert os.path.isfile(regions_to_plot_pls_csv), f"Could not find the file '{regions_to_plot_pls_csv}'"
        AllenBrain.select_from_csv(regions_to_plot_pls_csv, key="acronym")
    case s if s.startswith("depth"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{REGIONS_TO_PLOT_SELECTION_METHOD}' for REGIONS_TO_PLOT_SELECTION_METHOD")
regions_to_plot = AllenBrain.get_selected_regions()
print(f"You selected {len(regions_to_plot)} regions to plot.")

In [ ]:
for animal_group in animal_groups:
    animal_group.remove_smaller_subregions(MIN_AREA, regions_to_plot, AllenBrain)

In [ ]:
if MATRIX_SAVE_PLOT or MATRIX_SHOW_PLOT or CHORD_SAVE_PLOT or CHORD_SHOW_PLOT:
    groups_cross_correlations = [BraiAn.CrossCorrelation(g, regions_to_plot, AllenBrain, NORMALIZATION, MIN_ANIMALS_CROSS_CORRELATION, g.name) for g in animal_groups]
    
    if PLOT_ONLY_REGIONS_PRESENT_IN_ALL_GROUPS:
        BraiAn.CrossCorrelation.make_comparable(*groups_cross_correlations)
    if not PLOT_REGIONS_WITH_INSUFFICIENT_DATA:
        for cc in groups_cross_correlations:
            cc.remove_insufficient_regions()
    regions_to_plot_selection_method_str = REGIONS_TO_PLOT_SELECTION_METHOD.replace(".", "_").replace(" ", "_")

In [ ]:
if MATRIX_SAVE_PLOT or MATRIX_SHOW_PLOT:
    for group, cc in zip(animal_groups, groups_cross_correlations):
        title = f"{group.name} Pearson cross correlation matrix (n = {group.n})"
        fig = cc.plot(
                title=title,
                cell_height=MATRIX_CELL_HEIGHT, min_plot_height=MATRIX_MIN_PLOT_HEIGHT,
                star_size=MATRIX_STAR_SIZE, aspect_ratio=MATRIX_CELL_RATIO,
                color_min=-1, color_max=1
                )
        if MATRIX_SAVE_PLOT:
            plot_filename = f"correlation_matrix_min{MIN_ANIMALS_CROSS_CORRELATION}_{group.name}_{group.marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}{SAVED_PLOT_EXTENSION}".lower()
            plot_filepath = os.path.join(plots_output_path, plot_filename)
            match SAVED_PLOT_EXTENSION.lower():
                case ".html":
                    fig.write_html(plot_filepath, config=dict(toImageButtonOptions=dict(format="svg")))
                case _:
                    fig.write_image(plot_filepath)
        if MATRIX_SHOW_PLOT:
            fig.show()

In [ ]:
p_str = str(NETWORK_P_CUTOFF).replace(".", "_")
r_str = str(NETWORK_R_CUTOFF).replace(".", "_")
for animal_group, cc in zip(animal_groups, groups_cross_correlations):
    connectome = BraiAn.FunctionalConnectome(cc,
                                         p_cutoff=NETWORK_P_CUTOFF, r_cutoff=NETWORK_R_CUTOFF,
                                         negatives=NETWORK_USE_NEGATIVE_LINKS, isolated_vertices=True, weighted=True)
    if REGIONS_TO_PLOT_SELECTION_METHOD == "nre to bla" and "REtot" in cc.p.index:
        connectome = connectome.region_subgraph("REtot", isolated_vertices=True)
    title = f"{animal_group.name} connectomics graph from Pearson correlation (n = {animal_group.n}, {'|r|' if NETWORK_USE_NEGATIVE_LINKS else 'r'} >= {NETWORK_R_CUTOFF}, p <= {NETWORK_P_CUTOFF})"
    fig = BraiAn.draw_chord_plot(connectome,
                                AllenBrain=AllenBrain,
                                ideograms_arc_index=50,
                                title=title,
                                size=CHORD_PLOT_SIZE,
                                no_background=CHORD_NO_BACKGROUND,
                                regions_size=CHORD_REGIONS_SIZE,
                                regions_font_size=CHORD_REGIONS_FONT_SIZE,
                                max_edge_width=CHORD_MAX_EDGE_WIDTH,
                                use_weighted_edge_widths=CHORD_USE_WEIGHTED_EDGE_WIDTHS,
                                colorscale_edges=CHORD_USE_COLORSCALE_EDGES,
                                colorscale=CHORD_COLORSCALE,
                                colorscale_min=CHORD_COLORSCALE_MIN,
    )
    fig.show()
    # filename = f"chord_plot_min{MIN_ANIMALS_CROSS_CORRELATION}_p{p_str}_r{r_str}_{animal_group.name.lower()}_{animal_groups[0].marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}.html"
    # fig.write_html(filename.lower(), config=dict(toImageButtonOptions=dict(format="svg")))

In [ ]:
import igraph as ig

# https://doi.org/10.1109/TKDE.2007.190689
# clustering_args = (ig.Graph.community_optimal_modularity,)
                    # Graphs with up to fifty vertices should be fine, graphs with a couple of hundred vertices might be possible.
                    # crashes on my pc with summary structure's Graph
clustering_args = (ig.Graph.community_fastgreedy,)
# clustering_args = (ig.Graph.community_infomap,)
# clustering_args = (ig.Graph.community_leading_eigenvector_naive,)
# clustering_args = (ig.Graph.community_leading_eigenvector,)
# clustering_args = (ig.Graph.community_label_propagation,)
# clustering_args = (ig.Graph.community_multilevel,)
# clustering_args = (ig.Graph.community_edge_betweenness, dict(directed=False))
# clustering_args = (ig.Graph.community_spinglass,)
# clustering_args = (ig.Graph.community_walktrap,)
# clustering_args = (ig.Graph.community_leiden,)

In [ ]:
# layout_fun = ig.Graph.layout_kamada_kawai               # 1st best - good for ~small networks
layout_fun = ig.Graph.layout_fruchterman_reingold       # 2nd best - good for bigger networks
# layout_fun = ig.Graph.layout_graphopt                   # 3rd best
# layout_fun = ig.Graph.layout_davidson_harel             # 4th best
# layout_fun = ig.Graph.layout_mds                        # 5th best

In [ ]:
for animal_group, cc in zip(animal_groups, groups_cross_correlations):
    connectome = BraiAn.FunctionalConnectome(cc,
                                        p_cutoff=NETWORK_P_CUTOFF, r_cutoff=NETWORK_R_CUTOFF,
                                        negatives=NETWORK_USE_NEGATIVE_LINKS, isolated_vertices=NETWORK_USE_ISOLATED_VERTICES, weighted=False)
    if REGIONS_TO_PLOT_SELECTION_METHOD == "nre to bla" and "REtot" in cc.p.index:
        connectome = connectome.region_subgraph("REtot", isolated_vertices=NETWORK_USE_ISOLATED_VERTICES)
    connectome.cluster_regions(*clustering_args)
    
    title = f"{connectome.name} connectomics graph from Pearson correlation (n = {connectome.n}, {'|r|' if NETWORK_USE_NEGATIVE_LINKS else 'r'} >= {NETWORK_R_CUTOFF}, p <= {NETWORK_P_CUTOFF})"
    random.seed(0) # used by layout_fun to arrange the nodes of the connectome
    fig = BraiAn.draw_network_plot(connectome, layout_fun, AllenBrain, title=title)
    fig.show()
    # filename = f"network_min{MIN_ANIMALS_CROSS_CORRELATION}_p{p_str}_r{r_str}_{animal_group.name.lower()}_{animal_groups[0].marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}.html"
    # fig.write_html(filename.lower(), config=dict(toImageButtonOptions=dict(format="svg")))

In [ ]:
import numpy as np
import pandas as pd
# from https://download.alleninstitute.org/publications/A_high_resolution_data-driven_model_of_the_mouse_connectome/
normalized_connection_density_file = os.path.join(project_path, "data",
                                                    "A_high_resolution_data-driven_model_of_the_mouse_connectome",
                                                    "normalized_connection_density.csv")
BraiAn.cache(normalized_connection_density_file,
             "https://download.alleninstitute.org/publications/A_high_resolution_data-driven_model_of_the_mouse_connectome/normalized_connection_density.csv")

In [ ]:
def find_indices(where):
    rows = where.index[where.any(axis=1)]
    return [(row, col) for row in rows for col in where.columns if where.loc[row, col]]

# find_indices(normalized_connection_density == 0)

In [ ]:
log10_cutoff = -3.8
allen_connectome = BraiAn.StructuralConnectome(normalized_connection_density_file, regions_to_plot, AllenBrain, mode="max", log10_cutoff=log10_cutoff)
print(f"""
Max: {allen_connectome.A.max()}
Mean: {allen_connectome.A.mean()}+-{allen_connectome.A.std()}
Mean (log10): {allen_connectome.A.mean(log=True)}+-{allen_connectome.A.std(log=True)}
Density: {allen_connectome.G.density()}
""")

In [ ]:
# same as Figure 2 in https://direct.mit.edu/netn/article/3/1/217/2194/High-resolution-data-driven-model-of-the-mouse
allen_connectome.plot_adjacency(color_min=-5, color_max=-2.5, cell_height=4)\
                .show()

In [ ]:
re = allen_connectome.G.vs.select(name="RE")[0]
print("RE->BLA density:", allen_connectome.A.data.loc["RE", "BLA"], f"(log10={np.log10(allen_connectome.A.data.loc['RE', 'BLA'])})")
print("BLA->RE density:", allen_connectome.A.data.loc["BLA", "RE"], f"(log10={np.log10(allen_connectome.A.data.loc['BLA', 'RE'])})")
print("RE->ILA density:", allen_connectome.A.data.loc["RE", "ILA"], f"(log10={np.log10(allen_connectome.A.data.loc['RE', 'ILA'])})")
print("ILA->RE density:", allen_connectome.A.data.loc["ILA", "RE"], f"(log10={np.log10(allen_connectome.A.data.loc['ILA', 'RE'])})")
print(f"\nNetwork with log10(density) cutoff={log10_cutoff} ({10**log10_cutoff})")
print("\tRE Degree:", re.degree(), f"({re.indegree()}+{re.outdegree()})")
for e in re.all_edges():
    print(f"\t\t{e.source_vertex['name']}->{e.target_vertex['name']}: {e['weight']:.3f} ({10**e['weight']:.6f})")

In [ ]:
# allen_connectome.G.es["p-value"] = 0
# allen_connectome.G.vs["is_undefined"] = False
# allen_connectome.G.vs["upper_region"] = [allen_connection_matrix.upper_regions[region] for region in allen_connectome.G.vs["name"]]
# allen_connectome = BraiAn.FunctionalConnectome(None, None, r_cutoff=0, graph=G_allen, n=None, name="Allen's Connectome")
fig = BraiAn.draw_chord_plot(allen_connectome,
                            AllenBrain=AllenBrain,
                            ideograms_arc_index=50,
                            title=allen_connectome.name+f" [log10(normalized density) >= {log10_cutoff}]",
                            size=CHORD_PLOT_SIZE,
                            no_background=CHORD_NO_BACKGROUND,
                            regions_size=CHORD_REGIONS_SIZE,
                            regions_font_size=CHORD_REGIONS_FONT_SIZE,
                            max_edge_width=CHORD_MAX_EDGE_WIDTH,
                            use_weighted_edge_widths=False, #CHORD_USE_WEIGHTED_EDGE_WIDTHS,
                            colorscale_edges=CHORD_USE_COLORSCALE_EDGES,
                            colorscale=CHORD_COLORSCALE,
                            colorscale_min=10**log10_cutoff,
                            colorscale_max=10**(-2.5) #np.log10(allen_connection_matrix.A.max(axis=None))
)
fig.show()

In [ ]:
NETWORK_P_CUTOFF = 0.01
for cc in groups_cross_correlations:
    fc_pruned = BraiAn.PrunedConnectomics(allen_connectome, cc, NETWORK_R_CUTOFF, NETWORK_P_CUTOFF, isolated_vertices=False, weighted=True)
    title = f"{fc_pruned.name} [n = {cc.n}, {'|r|' if NETWORK_USE_NEGATIVE_LINKS else 'r'} >= {NETWORK_R_CUTOFF}, p <= {NETWORK_P_CUTOFF}, d >= {10**log10_cutoff}]"
    fc_pruned.G.vs["PageRank"] = fc_pruned.G.pagerank()
    fc_pruned.G.vs["Betweeness"] = fc_pruned.G.betweenness()
    fc_pruned.G.vs["Harmonic"] = fc_pruned.G.harmonic_centrality()
    fc_pruned.G.vs["Eigenvector"] = fc_pruned.G.evcent()
    random.seed(0) # used by layout_fun to arrange the nodes of the connectome
    fig = BraiAn.draw_network_plot(fc_pruned, layout_fun, AllenBrain, title=title, use_centrality=True, centrality_metric="Harmonic")
    # fig = BraiAn.draw_chord_plot(fc_pruned,
    #                             AllenBrain=AllenBrain,
    #                             ideograms_arc_index=50,
    #                             title=title,
    #                             size=CHORD_PLOT_SIZE,
    #                             no_background=CHORD_NO_BACKGROUND,
    #                             regions_size=CHORD_REGIONS_SIZE,
    #                             regions_font_size=CHORD_REGIONS_FONT_SIZE,
    #                             max_edge_width=CHORD_MAX_EDGE_WIDTH,
    #                             use_weighted_edge_widths=CHORD_USE_WEIGHTED_EDGE_WIDTHS,
    #                             colorscale_edges=CHORD_USE_COLORSCALE_EDGES,
    #                             colorscale=CHORD_COLORSCALE,
    #                             colorscale_min=NETWORK_R_CUTOFF, #CHORD_COLORSCALE_MIN,
    # )
    fig.show()
    # fig.write_html(f"chord_{fc_pruned.name.lower()}.html")
    # fig.write_html(f"{fc_pruned.name.lower()}_r{NETWORK_R_CUTOFF}_p{NETWORK_P_CUTOFF}_d{log10_cutoff}.html")

In [ ]:
re = fc_pruned.G.vs.select(name="VPL")[0]
for e in re.all_edges():
    d = e["weight"] # fc_pruned. .data.loc[e.source_vertex['name'], e.target_vertex['name']]
    p = cc.p.data.loc[e.source_vertex['name'], e.target_vertex['name']]
    print(d >= 10**log10_cutoff, 10**log10_cutoff, log10_cutoff)
    print(f"{e.source_vertex['name']}->{e.target_vertex['name']}: r={e['weight']:.3f} p={p:.4f} d={d:.5f} ({np.log10(d):.6f})")

In [ ]:
i = 1
connectome = BraiAn.FunctionalConnectome(groups_cross_correlations[i],
                                         p_cutoff=NETWORK_P_CUTOFF, r_cutoff=NETWORK_R_CUTOFF,
                                         negatives=NETWORK_USE_NEGATIVE_LINKS, isolated_vertices=NETWORK_USE_ISOLATED_VERTICES, weighted=False)
connectome_w = BraiAn.FunctionalConnectome(groups_cross_correlations[i],
                                         p_cutoff=NETWORK_P_CUTOFF, r_cutoff=NETWORK_R_CUTOFF,
                                         negatives=NETWORK_USE_NEGATIVE_LINKS, isolated_vertices=NETWORK_USE_ISOLATED_VERTICES, weighted=True)
sorted(list(zip(connectome.G.degree(), connectome.G.vs["name"])), key=lambda x: x[0], reverse=True)

In [ ]:
import igraph as ig
import plotly.graph_objects as go
import numpy as np

def centrality_barplot(G: ig.Graph, centrality_fun, n=20, name="", yaxis="y1", offsetgroup=1, **kwargs):
    sorted_by_centrality = sorted(G.vs.indices, key=lambda i: centrality_fun(G, i, **kwargs), reverse=True)
    selected_regions = G.vs[sorted_by_centrality[:n]]
    return go.Bar(
        x=selected_regions["name"],
        y=centrality_fun(G, selected_regions, **kwargs),
        name=name,
        yaxis=yaxis,
        offsetgroup=offsetgroup
    )

fig = go.Figure(
    data=[
        centrality_barplot(connectome.G, ig.Graph.degree, n=20, name="Degree", mode="all", loops=False, yaxis="y1", offsetgroup=1),
        centrality_barplot(connectome.G, ig.Graph.pagerank, n=20, name="PageRank", yaxis="y2", offsetgroup=2),
        centrality_barplot(connectome.G, ig.Graph.betweenness, n=20, name="Betweenness", yaxis="y3", offsetgroup=3),
#        centrality_barplot(connectome.G, ig.Graph.closeness, n=20, name="Closeness", yaxis="y4", offsetgroup=4), # closeness makes little to no sense to be used in a graph with isolated vertices
        centrality_barplot(connectome.G, ig.Graph.harmonic_centrality, n=20, name="Harmonic", yaxis="y2", offsetgroup=5)
    ],
    layout=dict(
        title="top 20 ranked regions",
        yaxis=dict(
            title="Degree",
            side="left",
        ),
        yaxis2=dict(
            title="PageRank",
            side="right",
            overlaying="y"
        ),
        yaxis3=dict(
            title="Betweenness",
            side="left",
            overlaying="y",
            #autoshift=False, shift=-100, anchor="free"
        ),
        yaxis4=dict(
            title="Closeness",
            side="right",
            overlaying="y",
            #autoshift=True, anchor="free"
        ),
    ))
fig.show()

In [ ]:
import igraph as ig
import plotly.graph_objects as go
import numpy as np

def get_ranks(G, centrality_fun, **kwargs):
    centrality_scores = np.flip(np.sort(np.nan_to_num(centrality_fun(G, **kwargs), copy=False, nan=0.0)))
    _, unique_ranks = np.unique(centrality_scores, return_index=True) # returns indices of the unique centrality scores (in ascending order)
    ranks_repetitions = unique_ranks[:-1]-unique_ranks[1:]
    ranks_repetitions = np.insert(ranks_repetitions, 0, len(centrality_scores)-unique_ranks[0])
    ranks = np.repeat(unique_ranks, ranks_repetitions)
    return centrality_scores, np.flip(ranks)

def centrality_barplot(G: ig.Graph, centrality_fun, name="", yaxis="y1", offsetgroup=1, **kwargs):
    sorted_by_centrality = sorted(G.vs, key=lambda v: centrality_fun(G, v.index, **kwargs), reverse=True)
    scores, ranks = get_ranks(connectome.G, centrality_fun, **kwargs)
    return go.Bar(
        x=[v["upper_region"] for v in sorted_by_centrality],
        # y=1/(ranks+1),
        y=1/np.sqrt(ranks+1),
        hovertext=[f"Region: {v['name']}<br>{name}: {centrality_fun(G, v, **kwargs):.5f}" for v in sorted_by_centrality],
        name=name,
        yaxis=yaxis,
        offsetgroup=offsetgroup
    )

fig = go.Figure(
    data=[
        centrality_barplot(connectome.G, ig.Graph.degree, loops=False, offsetgroup=1, name="Degree", mode="all"),
        centrality_barplot(connectome.G, ig.Graph.pagerank, name="PageRank", offsetgroup=2),
        centrality_barplot(connectome.G, ig.Graph.betweenness, name="Betweenness", offsetgroup=3),
#        centrality_barplot(connectome.G, ig.Graph.closeness, name="Closeness", offsetgroup=4), # closeness makes little to no sense to be used in a graph with isolated vertices
        centrality_barplot(connectome.G, ig.Graph.harmonic_centrality, name="Harmonic", offsetgroup=5)
    ],
    layout=dict(
        title="Regions ranks with different metrics (grouped by major divisions)",
        yaxis=dict(
            title="Importance Score",
            side="left",
        ),
    ))
fig.show()

In [ ]:
go.Figure(
    [go.Histogram(
        x=connectome.G.degree(),
        xbins=dict(start=0, size=2,)
    )],
#    layout=dict(bargap=0.1)
)

In [ ]:
import igraph as ig
import numpy as np

def path_length(G: ig.Graph, unreachable_nodes=True):
    if G.is_weighted():
        ds = np.asarray(G.distances(weights=np.abs(G.es["weight"])), dtype=float)
    else:
        ds = np.asarray(G.distances(), dtype=float)
    if unreachable_nodes:
        ds[ds == np.inf] = 0
        N = G.vcount()
        return ds.sum(axis=0) / (N-1)
    else:
        ds[ds == np.inf] = np.nan
        Ns = (~np.isnan(ds)).sum(axis=0, dtype=float)
        den = Ns - 1
        den[den == 0] = np.nan
        return np.nansum(ds, axis=0) / den

def average_path_length(G: ig.Graph, unreachable_nodes=True):
    # if unreachable_nodes=False, it's equal to igraph.Graph.average_path_length(unconn=True)
    if G.is_weighted():
        ds = np.asarray(G.distances(weights=np.abs(G.es["weight"])), dtype=float)
    else:
        ds = np.asarray(G.distances(), dtype=float)
    if unreachable_nodes:
        ds[ds == np.inf] = 0
        N = G.vcount()
        return ds.sum() / (N * (N - 1))
    else:
        ds = ds[ds != np.inf]
        return ds.sum() / (len(ds) - G.vcount())

# import networkx as nx

# Gnx = nx.Graph([])
# Gnx.add_node(1)
# nx.global_efficiency(Gnx) # <- returns 0
def nodal_efficiency(G: ig.Graph):
    # https://en.wikipedia.org/wiki/Efficiency_(network_science)
    N = G.vcount()
    if N == 0:
        raise ValueError("Empty graph")
    elif N == 1:
#        return np.full(N, 1, dtype=float)
        return np.full(N, 0, dtype=float)
    if G.is_weighted():
        ws = np.abs(1.0 / np.asarray(G.es["weight"], dtype=float))
        ds = np.asarray(G.distances(weights=ws), dtype=float)
    else:
        ds = np.asarray(G.distances(), dtype=float)
    np.fill_diagonal(ds, np.NaN)
    efficiency = 1 / ds
    np.fill_diagonal(efficiency, 0)
    ne = np.apply_along_axis(sum, 0, efficiency) / (N-1)
    return ne

def global_efficiency(G):
    if G.vcount() == 0:
        return np.nan
    return nodal_efficiency(G).mean()

def local_efficiency(G, zero_degree=True):
    local_efficiency_i = []
    for i in range(G.vcount()):
        G_i = G.induced_subgraph(G.neighbors(i), "create_from_scratch")
        global_efficiency_i = global_efficiency(G_i)
        local_efficiency_i.append(global_efficiency_i)
    if zero_degree:
        return np.nansum(np.asarray(local_efficiency_i)) / len(local_efficiency_i)
    else:
        return np.nanmean(np.asarray(local_efficiency_i))

In [ ]:
def print_stats(G: ig.Graph):
    print(f"""
    INFO:
        graph type: {'Weighted' if G.is_weighted() else 'Not weighted'}
        #regions: {G.vcount()}
        #connected regions: {len([d for d in G.degree() if d > 0])}
        Max degree: {G.maxdegree()}
    SEGREGATION:
        Cluster coefficient: {G.transitivity_undirected()}
        Mean local cluster coefficient (d>=2): {np.nanmean(G.transitivity_local_undirected())}
        Mean local cluster coefficient (all): {np.nansum(G.transitivity_local_undirected()) / G.vcount()}
        Local efficiency (d>=1): {local_efficiency(G, zero_degree=False)}
        Local efficiency (all): {local_efficiency(G, zero_degree=True)}
    INTEGRATION_
        Global efficiency: {global_efficiency(G)}
        Avg path length (∞ -> 0): {path_length(G, unreachable_nodes=True).mean()}
        Avg path length (no ∞): {G.average_path_length(unconn=True)}
        Median [characteristic] path lengh (∞ -> 0): {np.nanmedian(path_length(G, unreachable_nodes=True))}
        Median [characteristic] path lengh (no ∞): {np.nanmedian(path_length(G, unreachable_nodes=False))}
    """)

print_stats(connectome.G)
print_stats(connectome_w.G)

In [ ]:
# WARNING: certain measures of statistical dependence used to quantify functional connectivity can bias
# network organization in a way that cannot be removed by topological rewiring.
# 
G_w_rewired = connectome_w.G.copy()
G_w_rewired.rewire(mode="loops")    # functional connectomes are inherently more clustered and exaggerate features such as small-worldness.
                                    # We should use Maslov-Sneppen rewiring method (Bullmore, et al. 2016 - Fundamentals of Brain Network Analysis, chapter 10.3)
G_w_rewired.es["weight"] = connectome_w.G.es["weight"]
print_stats(G_w_rewired)

G_rewired = connectome.G.copy()
#G_rewired.rewire(mode="simple") # does not create/destroy loop edges. If you allow it, you may change the degrees
G_rewired.rewire(mode="loops")
print_stats(G_rewired)

In [ ]:
import importlib
__imported_modules = sys.modules.copy()
for module_name, module in __imported_modules.items():
    if not module_name.startswith("BraiAn"):
        continue
    try:
        importlib.reload(module)
    except ModuleNotFoundError:
        continue